# fairing.config.fn 예제

In [1]:
import tensorflow as tf

def train():
    mnist = tf.keras.datasets.mnist

    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    model.summary()

    print("Training...")

    model.fit(
        x_train, y_train, 
        epochs=3, 
        validation_split=0.2 
    ) 

    score = model.evaluate(x_test, y_test, batch_size=128, verbose=0)
    print('Test accuracy: ', score[1])

In [2]:
# Local training
train()

11493376/11490434 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Training...
Train on 48000 samples, validate on 12000 samples
Epoch 1/3
48000/48000 [==============================] - 6s 135us/sample - loss: 0.3202 - accuracy: 0.9081 - val_loss: 0.

In [3]:
import os
from kubeflow import fairing

fairing.config.set_builder(
    'append',
    base_image = 'tensorflow/tensorflow:latest-py3', 
    registry   = 'kubeflow-registry.default.svc.cluster.local:30000', 
    push=True
)

fairing.config.set_deployer('job')

remote_train = fairing.config.fn(train)
remote_train()

[I 201205 19:29:24 config:134] Using preprocessor: <kubeflow.fairing.preprocessors.function.FunctionPreProcessor object at 0x7f55b6f6c710>
[I 201205 19:29:24 config:136] Using builder: <kubeflow.fairing.builders.append.append.AppendBuilder object at 0x7f55b448f710>
[I 201205 19:29:24 config:138] Using deployer: <kubeflow.fairing.deployers.job.job.Job object at 0x7f55b6f6c908>
[W 201205 19:29:24 append:50] Building image using Append builder...
[I 201205 19:29:24 base:107] Creating docker context: /tmp/fairing_context_w9b5qcrf
[W 201205 19:29:24 base:94] /usr/local/lib/python3.6/dist-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
[I 201205 19:29:24 docker_creds_:234] Loading Docker credentials for repository 'tensorflow/tensorflow:latest-py3'
[W 201205 19:29:27 append:54] Image successfully built in 2.372721273000934s.
[W 201205 19:29:27 append:94] Pushing image kubeflow-registry.default.svc.cluster.local:30000/fairing-job:223E3522...
[I 201205 19:2

hello world!
Bye!


[W 201205 19:29:40 job:173] Cleaning up job fairing-job-46fdb...


### curl로 프라이빗 레지스트리 확인

In [4]:
! curl http://kubeflow-registry.default.svc.cluster.local:30000/v2/_catalog

{"repositories":["covid19-katib-job","fairing-job","katib-base","katib-job","kf-base","tensorboard-job"]}


In [5]:
! curl http://kubeflow-registry.default.svc.cluster.local:30000/v2/fairing-job/tags/list

{"name":"fairing-job","tags":["3DA1A245","18DF7E85","223E3522","141D2F60","8E296311","66787D3","3052981C"]}
